# Extracting bitcoin data from http://www.coingecko.com
## Using pycoingecko API Library
### Learning simple project

## [1] Install and import pycoingecko API library

In [1]:
!pip install pycoingecko

from pycoingecko import CoinGeckoAPI

  Using cached pycoingecko-2.2.0-py3-none-any.whl (8.3 kB)


## [2] Creat a client object:

In [2]:
cg = CoinGeckoAPI()

## [3] Use a function to request your data: 

In [3]:
bitcoin_data = cg.get_coin_market_chart_by_id(id='bitcoin', vs_currency='usd', days=30)

# now we got our bitcoin data in usd currency for the last 30 days in the form of JSON file (Dictionary data)
# This data contains prices, volumes and others so we need to extract the data we need which is price

## [4] Extract price data from your dictionary

In [4]:
print(bitcoin_data.keys())

bitcoin_price_data = bitcoin_data['prices'] 

print(type(bitcoin_price_data))             ## now we got our data in the form of nested list

print(len(bitcoin_price_data[0]))

dict_keys(['prices', 'market_caps', 'total_volumes'])
<class 'list'>
2


## [5] Convert bitcoin_price_data '*nested list*' to dataframe:

In [5]:
#!pip install pandas
import pandas as pd

data = pd.DataFrame(bitcoin_price_data, columns=['time stamp', 'price'])

print(data.head())

      time stamp         price
0  1628053255507  38175.250805
1  1628056952367  38130.537628
2  1628060408957  38162.259996
3  1628064062118  37781.150052
4  1628067607241  37729.027675


## [6] Convert time stamp data into readable format: *by adding a date column and convert time stamp column*

In [6]:
#!pip install datetime
import datetime

data['Date'] = pd.to_datetime(data['time stamp'], unit='ms')

print(data.head())

      time stamp         price                    Date
0  1628053255507  38175.250805 2021-08-04 05:00:55.507
1  1628056952367  38130.537628 2021-08-04 06:02:32.367
2  1628060408957  38162.259996 2021-08-04 07:00:08.957
3  1628064062118  37781.150052 2021-08-04 08:01:02.118
4  1628067607241  37729.027675 2021-08-04 09:00:07.241


## [7] Create a candlestick plot for your data

In [7]:
# To get the data ready for candlestick plot we need to group our data by day and to get min, max, first and last price of the days

candlestick_data = data.groupby(data.Date.dt.date).agg({'price': ['min', 'max', 'first', 'last']})

print(candlestick_data.head())

                   price                                          
                     min           max         first          last
Date                                                              
2021-08-04  37729.027675  39877.209001  38175.250805  39653.344554
2021-08-05  37595.746824  40952.350810  39756.280965  40927.234106
2021-08-06  39871.571561  43133.328488  40825.381940  42667.089957
2021-08-07  42813.247737  44114.342504  42813.247737  43979.139208
2021-08-08  43572.946706  45157.582576  44514.605515  44396.813230


## [8] Use plotly to create candlestick chart:

In [8]:

#!pip install plotly
#!pip install matplotlib
#!pip install mplfinance

import plotly.graph_objects as go
from plotly.offline import plot
import matplotlib.pyplot as plt
from mplfinance.original_flavor import candlestick2_ohlc

fig = go.Figure(data=[go.Candlestick(x= candlestick_data.index, open= candlestick_data['price', 'first'], high= candlestick_data['price', 'max'], low= candlestick_data['price', 'min'], close= candlestick_data['price', 'last'])])

fig.update_layout(xaxis_rangeslider_visible=False, xaxis_title='Date', yaxis_title='Price (USD $)', title= 'Bitcoin Candlestick Chart Over Past 30 Days')

fig.show()
#plot(fig, filename='bitcoin_candlestick_graph.html')            # Use this line of code if you want to shoe chart as html